In [175]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [10]:
df = pd.read_csv('Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [26]:
# df.drop(['customerID','gender','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies'],axis=1,inplace=True)

In [11]:
df.drop(columns=['gender', 'PhoneService'], axis=1, inplace=True)

In [14]:
le = LabelEncoder()

def encode(x):
    if (x.dtype == 'object'):
        x = le.fit_transform(x)
    return x
    
df = df.apply(lambda x: encode(x))

In [15]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [16]:
df.dropna(inplace=True)

In [18]:
df.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,0,0,1,1,0,0,2,0,0,0,0,0,1,2,29.85,2505,0
1,3962,0,0,0,34,0,0,2,0,2,0,0,0,1,0,3,56.95,1466,0
2,2564,0,0,0,2,0,0,2,2,0,0,0,0,0,1,3,53.85,157,1
3,5535,0,0,0,45,1,0,2,0,2,2,0,0,1,0,0,42.30,1400,0
4,6511,0,0,0,2,0,1,0,0,0,0,0,0,0,1,2,70.70,925,1


In [19]:
df = pd.get_dummies(df, columns=['MultipleLines','InternetService','Contract','PaymentMethod'], drop_first=True, dtype=np.float32)

In [20]:
df.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,...,Churn,MultipleLines_1,MultipleLines_2,InternetService_1,InternetService_2,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,5375,0,0,0,1,0,2,0,0,0,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3962,0,0,0,34,2,0,2,0,0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2564,0,0,0,2,2,2,0,0,0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5535,0,0,0,45,2,0,2,2,0,...,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,6511,0,0,0,2,0,0,0,0,0,...,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
scaler = StandardScaler()

X = df.drop('Churn',axis=1)
for col in X.columns:
    X[col] = scaler.fit_transform(X[col].values.reshape(-1,1))
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [313]:

class myModel(nn.Module):
    def __init__(self, features):
        super().__init__()
        self.fc1 = nn.Linear(features, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.out = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        return F.sigmoid(self.out(x))

In [324]:
class myModelComplex(nn.Module):
    def __init__(self, features):
        super().__init__()
        self.fc1 = nn.Linear(features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = nn.Linear(1024, 256)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.fc3 = nn.Linear(256, 512)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.fc4 = nn.Linear(512, 128)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        
        self.out = nn.Linear(64, 1)
        
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        return self.out(x)

In [325]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

batch_size=256

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [326]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [327]:
features = train_data.tensors[0].shape[1]

# model = myModelComplex(features) BCE logits
model = myModel(features)
model = model.to(device)

lr = 1e-4
optim = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
scheduler = StepLR(optim, step_size=500, gamma=0.1)
# scheduler = ReduceLROnPlateau(optim, mode='max', factor=0.5, patience=100, verbose=True, threshold=0.0005)


# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.BCELoss()

lossi = []

In [328]:
def accuracy():
    model.eval()
    correct_count = 0.0

    for X_test, y_test in test_loader:
        X_test = X_test.to(device)
        y_test = y_test.to(device)
        pred = model(X_test)
        pred = pred.view(-1)
        out = (pred > 0.5).float()
        correct_count += (y_test == out).sum()
        
    return correct_count.item()/len(test_loader.dataset)

In [340]:
num_epoch = 1000

for ep in range(num_epoch):
    model.train()
    running_loss = 0.0

    for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optim.zero_grad()
            outputs = model(X).squeeze()
            loss = loss_fn(outputs, y.float())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optim.step()
            running_loss += loss.item()
            
            with torch.no_grad():
                lossi.append(loss.item())
    
    test_acc = accuracy()
    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    if(ep%100 == 0):
        print(f"Epoch {ep+1}/{num_epoch}, Lr:{scheduler.get_last_lr()[-1]}, Loss: {epoch_loss / len(train_loader):.8f}, Accuracy: {test_acc:.4f}")

Epoch 1/1000, Lr:1.0000000000000006e-10, Loss: 0.01660234, Accuracy: 0.7984
Epoch 101/1000, Lr:1.0000000000000006e-10, Loss: 0.01610668, Accuracy: 0.7907
Epoch 201/1000, Lr:1.0000000000000006e-10, Loss: 0.01608995, Accuracy: 0.8217
Epoch 301/1000, Lr:1.0000000000000006e-10, Loss: 0.01605469, Accuracy: 0.8295
Epoch 401/1000, Lr:1.0000000000000006e-10, Loss: 0.01591643, Accuracy: 0.8062
Epoch 501/1000, Lr:1.0000000000000006e-11, Loss: 0.01640252, Accuracy: 0.8062
Epoch 601/1000, Lr:1.0000000000000006e-11, Loss: 0.01558534, Accuracy: 0.7984
Epoch 701/1000, Lr:1.0000000000000006e-11, Loss: 0.01592364, Accuracy: 0.7984
Epoch 801/1000, Lr:1.0000000000000006e-11, Loss: 0.01618856, Accuracy: 0.8062
Epoch 901/1000, Lr:1.0000000000000006e-11, Loss: 0.01638756, Accuracy: 0.8217


In [344]:
acc = accuracy()

print(f'Accuracy: {acc:.3f}')

Accuracy: 0.829
